In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# Define the path to your JSON file
file_path = '/content/drive/MyDrive/MasterThesis/captions_data_combined_all.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)


In [ ]:
!pip install evaluate transformers

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the necessary components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def summarize_captions(captions):
    combined_text = ' '.join(captions)
    input_length = len(combined_text.split())  # Calculate the input length

    # Dynamically adjust max_length with a gentler slope, ensuring it doesn't exceed 75% of the input length
    # but is also not shorter than a minimum viable length for coherence
    max_length_ratio = 0.75
    min_viable_length = 20  # Adjust this as needed to ensure summaries are not too short
    max_length = max(min_viable_length, int(input_length * max_length_ratio))

    # Ensure min_length is logical given the new max_length, aiming for summaries to have enough room to be coherent
    min_length = max(10, max_length // 2)

    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

iter=0
# Process each item in the JSON
for key, item in captions_data.items():
    # Filter out to only include captions from gpt2 and blip
    generated_captions = [caption for source, caption in item['generated_captions'].items() if source in ['gpt2', 'blip']]

    # Check if there are captions to summarize
    if generated_captions:
        summarized_caption = summarize_captions(generated_captions)

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure
        captions_data[key]['summarized_caption'] = summarized_caption
        captions_data[key]['most_similar_caption'] = most_similar_caption
        captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

        # Optional: Print the information for verification
        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("="*50)
        print(iter)
    else:
        print(f"No captions from 'gpt2' or 'blip' to summarize for item {key}.")

    iter+=1

# Save the modified JSON data to a new file
output_path = '/content/drive/MyDrive/MasterThesis/Scenario1_distilbart_with_all_similarities_only_2.json'  # Update this path
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


Summarized Caption:  a person in a black hoodie a hot dog on a bun with mustard and k
Most Similar Original Caption: some food is sticking out of a toy car
Cosine Similarity Score: 0.6095834374427795
0
Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarity Score: 0.8367483019828796
1
Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarity Score: 0.8367483019828796
2
Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarity Score: 0.8367483019828796
3
Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarit

Your max_length is set to 20, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarized Caption:  A number of people riding scoots on a street a number of motorcycle riders on a
Most Similar Original Caption: Several people who are riding motor scooters, stopped at an intersection.
Cosine Similarity Score: 0.7803629040718079
6
Summarized Caption:  A dog laying in a dirty bathroom is a dog laying on the ground in a bathroom
Most Similar Original Caption: A dog is laying on a dirty bathroom floor.
Cosine Similarity Score: 0.9529235363006592
7
Summarized Caption:  A man riding a bike on a city street is seen riding on a sidewalk next to
Most Similar Original Caption: A person riding a bicycle on a deserted street.
Cosine Similarity Score: 0.8782333731651306
8
Summarized Caption:  A bathroom with marble walls and a large mirror is a bathroom with a sink, mirror
Most Similar Original Caption: A bathroom with marble walls and counters surround a large mirror.
Cosine Similarity Score: 0.9143549203872681
9


Your max_length is set to 20, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarized Caption:  a dog standing in a bathroom with a toilet a dog is standing in the bathroom with
Most Similar Original Caption: A brown dog standing next to a toilet in a bathroom.
Cosine Similarity Score: 0.8888086080551147
10


...

Summarized Caption:  a man kneeling on the floor in front of a refrigerator a young girl is looking at
Most Similar Original Caption: A man kneeling down in front of a refrigerator.
Cosine Similarity Score: 0.890028715133667
1080
Summarized Caption:  a stone counter a kitchen with a large counter top and a large counters top is a
Most Similar Original Caption: Chefs work in an open kitchen and at a counter.
Cosine Similarity Score: 0.829271674156189
1081
Summarized Caption:  a kitchen with a table and chairs in it is a kitchen without a table or chairs
Most Similar Original Caption: An empty kitchen and dining room during the day.
Cosine Similarity Score: 0.8714486956596375
1082
Summarized Caption:  a circular picture of a living room with a staircase a kitchen with a lot of clutter
Most Similar Original Caption: A circular view of an area that features a woman and kitchen counters and shelving.
Cosine Similarity Score: 0.7410973310470581
1083


Your max_length is set to 20, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarized Caption:  a picnic table with a bunch of backpacks on it has a lot of camping gear
Most Similar Original Caption: A picnic table at a campsite is loaded with tons of camping gear.
Cosine Similarity Score: 0.9035295844078064
1084


Your max_length is set to 20, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Summarized Caption:  A wooden table in a kitchen is a kitchen with a table and a bowl of fruit
Most Similar Original Caption: A bowl of fruit is placed on a table in a kitchen.
Cosine Similarity Score: 0.8555423617362976
1085
Summarized Caption:  A woman preparing food a man in a kitchen cutting up food . A man preparing food
Most Similar Original Caption: He is working on the meal while the dog is patiently waiting. 
Cosine Similarity Score: 0.8101833462715149
1086
Summarized Caption:  A kitchen with a sink and a stove is a kitchen without a stove, sink,
Most Similar Original Caption: A kitchen with a lot of items sitting out. 
Cosine Similarity Score: 0.8436746597290039
1087
Summarized Caption:  a man standing in a kitchen with a dog a kitchen  with a stove, a
Most Similar Original Caption: A man points at the wall in his kitchen.
Cosine Similarity Score: 0.7602767944335938
1088
Summarized Caption:  A bathroom with a sink and mirror in it a bathroom has a sink, mirror,
Most Similar O

In [ ]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the necessary components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Function to clean captions
def clean_caption(caption):
    return caption.replace("\\", "").replace("\"", "")

# Compute cosine similarities for generated caption vs. original captions
def compute_cosine_similarities(generated_caption, original_captions, clip_processor, clip_model):
    inputs_generated = clip_processor(text=[generated_caption], return_tensors="pt", truncation=True, padding=True)
    embeddings_generated = clip_model.get_text_features(**inputs_generated)

    similarities = []
    for orig_caption in original_captions:
        inputs_orig = clip_processor(text=[orig_caption], return_tensors="pt", truncation=True, padding=True)
        embeddings_orig = clip_model.get_text_features(**inputs_orig)
        cosine_similarity_score = torch.cosine_similarity(embeddings_generated, embeddings_orig, dim=1).cpu().item()
        similarities.append(cosine_similarity_score)

    return similarities

iter=0
# Process each item in the JSON
for key, item in captions_data.items():

    # Clean and summarize generated captions
    generated_captions = [clean_caption(caption) for caption in list(item['generated_captions'].values())]
    original_captions = item['original_coco_captions']
    summarized_caption = summarize_captions(generated_captions)
    print(summarized_caption)

    # Compute and store similarities for each model vs. original captions
    for model_name in ['blip', 'gpt2', 'pix2struct']:
        item[f"{model_name}_vs_Orig"] = compute_cosine_similarities(item['generated_captions'][model_name], original_captions, clip_processor, clip_model)

    # Compute similarities between summarized caption and original captions
    summarized_vs_orig_similarities = compute_cosine_similarities(summarized_caption, original_captions, clip_processor, clip_model)
    most_similar_index = summarized_vs_orig_similarities.index(max(summarized_vs_orig_similarities))
    most_similar_caption = original_captions[most_similar_index]
    cosine_similarity_score = summarized_vs_orig_similarities[most_similar_index]

    # Optional: Print for verification
    print(f"Summarized Caption: {summarized_caption}")
    print(f"Most Similar Original Caption: {most_similar_caption}")
    print(f"Cosine Similarity Score: {cosine_similarity_score}\n" + "="*50)
    print(iter)
    iter+=1
    if iter==3:
      break

# Save the modified JSON data to a new file
output_path = '/content/drive/MyDrive/MasterThesis/Scenario1_distilbart_with_all_similarities_1.json'  # Update this path
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")

Your max_length is set to 130, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


 A person in a black hoodie a hot dog on a bun with mustard and ketchup . A man is holding a book titled The Human Family .


Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Summarized Caption:  A person in a black hoodie a hot dog on a bun with mustard and ketchup . A man is holding a book titled The Human Family .
Most Similar Original Caption: A plate that has a cardboard car holding a hot dog and fries in it.
Cosine Similarity Score: 0.4990827143192291
0
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table . A bottle of Coke sits next to a glass of beer .


Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table . A bottle of Coke sits next to a glass of beer .
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarity Score: 0.7961562275886536
1
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table . A bottle of Coke sits next to a glass of beer .
Summarized Caption:  a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table . A bottle of Coke sits next to a glass of beer .
Most Similar Original Caption: A picture of two hot dogs and a cup of soda
Cosine Similarity Score: 0.7961562275886536
2
JSON file has been updated and saved.


In [ ]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the necessary components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Define the function to summarize captions
def summarize_captions(captions):
    combined_text = ' '.join(captions)
    input_length = len(combined_text.split())  # Calculate the input length

    # Dynamically adjust max_length with a gentler slope, ensuring it doesn't exceed 75% of the input length
    # but is also not shorter than a minimum viable length for coherence
    max_length_ratio = 0.75
    min_viable_length = 20  # Adjust this as needed to ensure summaries are not too short
    max_length = max(min_viable_length, int(input_length * max_length_ratio))

    # Ensure min_length is logical given the new max_length, aiming for summaries to have enough room to be coherent
    min_length = max(10, max_length // 2)

    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

# Function to clean captions
def clean_caption(caption):
    return caption.replace("\\", "").replace("\"", "")

# Compute cosine similarities for generated caption vs. original captions
def compute_cosine_similarities(generated_caption, original_captions, clip_processor, clip_model):
    inputs_generated = clip_processor(text=[generated_caption], return_tensors="pt", truncation=True, padding=True)
    embeddings_generated = clip_model.get_text_features(**inputs_generated)

    similarities = []
    for orig_caption in original_captions:
        inputs_orig = clip_processor(text=[orig_caption], return_tensors="pt", truncation=True, padding=True)
        embeddings_orig = clip_model.get_text_features(**inputs_orig)
        cosine_similarity_score = torch.cosine_similarity(embeddings_generated, embeddings_orig, dim=1).cpu().item()
        similarities.append(cosine_similarity_score)

    return similarities

iter=0
# Process each item in the JSON
for key, item in captions_data.items():

    # Clean and summarize generated captions
    generated_captions = [clean_caption(caption) for caption in list(item['generated_captions'].values())]
    original_captions = item['original_coco_captions']
    summarized_caption = summarize_captions(generated_captions)
    print(summarized_caption)

    # Compute and store similarities for each model vs. original captions
    for model_name in ['blip', 'gpt2']:
        item[f"{model_name}_vs_Orig"] = compute_cosine_similarities(item['generated_captions'][model_name], original_captions, clip_processor, clip_model)

    # Compute similarities between summarized caption and original captions
    summarized_vs_orig_similarities = compute_cosine_similarities(summarized_caption, original_captions, clip_processor, clip_model)
    most_similar_index = summarized_vs_orig_similarities.index(max(summarized_vs_orig_similarities))
    most_similar_caption = original_captions[most_similar_index]
    cosine_similarity_score = summarized_vs_orig_similarities[most_similar_index]

    # Update the JSON structure
    captions_data[key]['summarized_caption'] = summarized_caption
    captions_data[key]['most_similar_caption'] = most_similar_caption
    captions_data[key]['cosine_similarity_score'] = cosine_similarity_score

    # Optional: Print for verification
    print(f"Summarized Caption: {summarized_caption}")
    print(f"Most Similar Original Caption: {most_similar_caption}")
    print(f"Cosine Similarity Score: {cosine_similarity_score}\n" + "="*50)
    print(iter)
    iter+=1

# Save the modified JSON data to a new file
output_path = '/content/drive/MyDrive/MasterThesis/Scenario1_distilbart_with_all_similarities_only_2.json'  # Update this path
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")

Your max_length is set to 130, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


 A person in a black hoodie a hot dog on a bun with mustard and ketchup . A man is holding a book titled The Human Family .


Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Summarized Caption:  A person in a black hoodie a hot dog on a bun with mustard and ketchup . A man is holding a book titled The Human Family .
Most Similar Original Caption: A plate that has a cardboard car holding a hot dog and fries in it.
Cosine Similarity Score: 0.4990827143192291
0
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table . A bottle of Coke sits next to a glass of beer .


Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


...

Summarized Caption:  a plate of food with a bowl of soup and a sandwich on a plate . a bottle of beer next to a glass of beer . A plate of soup with a sandwich and a plate  with a glass  of beer on a glass . A bowl of soup and a sandwich with a  glass of beer on a plate.
Most Similar Original Caption: A plate that has a sandwich and bowl of soup on it.
Cosine Similarity Score: 0.827160120010376
0
 A kitchen with a fridge that says'stove' on it. a bathroom with a sink and a mirror . a bathroom . with sink, toilet and bathtub . A kitchen that says 'I'm not a man' with a refrigerator that says "I'm a woman"


Your max_length is set to 130, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Summarized Caption:  A kitchen with a fridge that says'stove' on it. a bathroom with a sink and a mirror . a bathroom . with sink, toilet and bathtub . A kitchen that says 'I'm not a man' with a refrigerator that says "I'm a woman"
Most Similar Original Caption: a bathroom that is stylish with a walk in shower
Cosine Similarity Score: 0.7598050832748413
0
 a white tiled bathroom with a toilet and a shower stall is a bathroom with toilet and shower stall . a black and white photo of a building with a sign that says The Building. a bathroom without a toilet or shower stall. a toilet with a sink and a sink .


Your max_length is set to 130, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


Summarized Caption:  a white tiled bathroom with a toilet and a shower stall is a bathroom with toilet and shower stall . a black and white photo of a building with a sign that says The Building. a bathroom without a toilet or shower stall. a toilet with a sink and a sink .
Most Similar Original Caption: Shot inside a plain bathroom from entrance with toilet nearest door.
Cosine Similarity Score: 0.8483728170394897
0
